In [ ]:
path = 'C:/Users/1/attribution_project/'

In [ ]:
m = 10
winning_algorithm = 'CatBoostRegressor' #one of: 'RandomForestRegressor','LinearRegression','XGBRegressor', 'LGBMRegressor', 'CatBoostRegressor'
winning_encoder = 'None' #one of: 'OneHotEncoder','CatBoostEncoder','None'

In [ ]:
algorithms = [winning_algorithm]
encoders = [winning_encoder]

Defining functions for interpretation

In [ ]:
import pandas as pd
import numpy as np
import itertools
import matplotlib as plt
import seaborn
import pickle

In [ ]:
def d_intraMedianAll(df):
    categories = list(df['category'].unique())
    l_intra = []
    for c in categories:
        s = list(df.loc[df['category']==c, 'SHAP'])
        l_intra.append(np.median(s))

    return l_intra

def d_intra(s):
    c = list(itertools.combinations(s,2))
    
    m = []
    for i in c:
        m.append(abs(i[1]-i[0]))
    if m == []:
        m = [0]
        
    return(np.median(m))

def d_intraAll(df):
    categories = list(df['category'].unique())
    l_intra = []
    for c in categories:
        s = list(df.loc[df['category']==c, 'SHAP'])
        l_intra.append(d_intra(s))

    return l_intra

def d_inter(df):
    categories = list(df['category'].unique())
    
    l_l_per_cat = []
    for c in categories:
        l_l_per_cat.append(list(df.loc[df['category']==c, 'SHAP']))
    
    l_l_pairs = list(itertools.combinations(l_l_per_cat,2))
    
    inter_meadians_per_cat = []
    inter_means_per_cat = []
    
    for pair in l_l_pairs:
        c = list(itertools.product(pair[0], pair[1]))
        
        m = []
        for i in c:
            m.append(abs(i[1]-i[0]))
    
        inter_meadians_per_cat.append(np.median(m))
        inter_means_per_cat.append(np.mean(m))        
    
    return inter_meadians_per_cat

In [ ]:
def global_SHAP(shap_v,df_v, metric='l_max_inter'):
    
    one_cat_vars = []
    
    l_intra_per_cat = []
    l_inter_per_cat = []
    l_max_intra = []
    l_max_inter = []
    l_inter_to_intra = []
    l_intra_medians_per_cat = []
    l_max_intra_median = []
    
    feature_list = df_v.columns
    
    for i in feature_list:
        df_feature = pd.concat([df_v[i],shap_v[i]],axis=1)
        df_feature.columns  = ['category','SHAP']
        
        #print(df_feature)
        
        intra_medians_per_cat = d_intraMedianAll(df_feature)
        
        intra_per_cat = d_intraAll(df_feature)
        inter_per_cat = d_inter(df_feature)
        
        max_intra_median = np.max(intra_medians_per_cat )
        max_intra = np.max(intra_per_cat)
        
        if len(inter_per_cat) > 0:
            max_inter = np.max(inter_per_cat)
            inter_to_intra = max_inter/max_intra
        else:
            print('one category variable:', i)
            print('category:', df_feature['category'].unique())
            one_cat_vars.append(i)
            max_inter = np.NaN
            inter_to_intra = np.NaN
        
        
        l_intra_per_cat.append(intra_per_cat)
        l_inter_per_cat.append(inter_per_cat)
        l_max_intra.append(max_intra)
        l_max_inter.append(max_inter)
        l_inter_to_intra.append(inter_to_intra)
        l_intra_medians_per_cat.append(intra_medians_per_cat) 
        l_max_intra_median.append(max_intra_median) 
        
        
    df_res = pd.concat([pd.Series(feature_list), pd.Series(l_intra_medians_per_cat), 
                        pd.Series(l_max_intra_median), pd.Series(l_intra_per_cat),
                        pd.Series(l_inter_per_cat),pd.Series(l_max_intra),pd.Series(l_max_inter),
                        pd.Series(l_inter_to_intra)],axis=1)#.fillna(0)

    df_res.columns  = ['Variable', 'intra_medians_per_cat', 'l_max_intra_median', 'intra_per_cat',
                       'l_inter_per_cat', 'l_max_intra', 'l_max_inter', 'inter_to_intra']
    
    df_res['Sign'] = np.where(df_res['inter_to_intra']<=1,'red','green')
    
    #drop one category variables
    df_res = df_res[~df_res['Variable'].isin(one_cat_vars)]
    
    # Plot it
    df_res = df_res.sort_values(by=metric, ascending = True)
    colorlist = df_res['Sign']
    
    font = {'weight' : 'bold',
            'size'   : 65}

    plt.rc('font', **font)
    
    ax = df_res.plot.barh(x='Variable',y=metric,color = colorlist, figsize=(50,60),legend=False)
    ax.set_xlabel(metric, fontsize=32)
    ax.set_ylabel("Variable", fontsize=75)
    
    return (df_res, one_cat_vars)

In [ ]:
def SHAP_bars(df_shap,df, one_cat_vars, selected, abs_vals=False):
    
    shap_v = pd.DataFrame(df_shap)
    feature_list = df.columns
    shap_v.columns = feature_list
    df_v = df.copy().reset_index().drop('index',axis=1)
    
    corr_df = pd.DataFrame(feature_list)
    corr_df.columns  = ['Variable']
    corr_df['Sign'] = np.where(corr_df['Variable'].isin(selected),'grey','blue')
    corr_df['Sign'] = np.where(corr_df['Variable'].isin(one_cat_vars),'grey', corr_df['Sign'])
    
    # Plot it
    if abs_vals:
        shap_abs = np.abs(shap_v)
    else:
        shap_abs = shap_v
        
    font = {'weight' : 'bold',
            'size'   : 65}
        
    k=pd.DataFrame(shap_abs.median()).reset_index()
    k.columns = ['Variable','SHAP_abs']
    k2 = k.merge(corr_df,left_on = 'Variable',right_on='Variable',how='inner')
    k2 = k2.sort_values(by='SHAP_abs',ascending = True)
    colorlist = k2['Sign']
    ax = k2.plot.barh(x='Variable',y='SHAP_abs',color = colorlist, figsize=(50,60),legend=False)
    if abs_vals:
        ax.set_xlabel("Median absolute SHAP value")
    else:
        ax.set_xlabel("Median SHAP Value")

In [ ]:
def show_categories(feature, df_v, shap_v):
    
    df_feature = pd.concat([df_v[feature],shap_v[feature]],axis=1)
    df_feature = pd.concat([df_v[feature],shap_v[feature]],axis=1)
    df_feature.columns  = ['category','SHAP']
    
    b = seaborn.boxplot(data=df_feature, x='category', y='SHAP')
    b.set_xlabel(feature)
    b.set_ylabel("SHAP value")
    b.tick_params()


In [ ]:
def choose_category(feature,category,selected,expected_value,df_v,shap_v):
    df_v = df_v[df_v[feature]==category]
    i1 = df_v.index
    
    i2 = shap_v.index
    shap_v = shap_v[i2.isin(i1)]

    selected.append(feature)
    
    expected_value += np.mean(shap_v[feature])

    df_v = df_v.drop(feature, axis=1).reset_index(drop=True)
    shap_v = shap_v.drop(feature, axis=1).reset_index(drop=True)

    return (selected, expected_value, df_v,shap_v)

In [ ]:
def shap(algorithm, encoder, path, dataset='test', shapley_library='shap'):
    
    import pickle
    
    import matplotlib.pyplot as plt
    import numpy as np
    import pandas as pd
    
    import shap
    if shapley_library == 'shap':
        pass
    elif shapley_library == 'acv':
        from acv_explainers import ACVTree
    
    path_in = path + 'output/' + algorithm + '_' + encoder + '_'
    path_out = path + 'output/' + library +'/'+ dataset + '/' + algorithm + '_' + encoder + '_'
    path_data = path + 'data/'
    
    #load data
    model = pickle.load(open(path_in + 'model.pkl','rb'))

    X = pickle.load(open(path_in + 'X_' + dataset + '.pkl','rb'))
    y = pickle.load(open(path_data + 'y_' + dataset + '.pkl','rb'))
    y_pred = pickle.load(open(path_in + 'y_' + dataset + '_pred.pkl','rb'))
    
    if encoder not in encoders_1to1:
        enc_list_col_ids = pickle.load(open(path_in + 'enc_list_col_ids.pkl','rb'))
        enc = pickle.load(open(path_in + 'enc.pkl','rb'))
        
    X_orig = pickle.load(open(path_data + 'X_' + dataset + '.pkl','rb'))
    
    #reset indexes
    X=X.reset_index(drop=True)
    y=y.reset_index(drop=True)
    X_orig=X_orig.reset_index(drop=True)
    
    #X_display - from orig, but integers repalicn categories
    X_display = X_orig.copy()
    
    #all dependent variables are categorical
    categorical_vars = X_orig.columns

    categories = {}
    encoding = {}
    decoding = {}

    for var in categorical_vars:
        categories[var] = list(X_display[var].unique())
        encoding[var] = {k: v for v, k in enumerate(categories[var])}
        decoding[var] = {v: k for k, v in encoding[var].items()}
    
        X_display[var] = X_display[var].map(encoding[var]).astype('int')
    
    if shapley_library=='shap':
        #fit the SHAP model
        shap.initjs()
        if algorithm == 'LinearRegression':
            X_train = pickle.load(open(path_in + 'X_train.pkl','rb'))
            explainer = shap.LinearExplainer(model, X_train)
        else:
            explainer = shap.TreeExplainer(model)
        shap_values = explainer.shap_values(X)
    
        #transforming for the SHAP model
        shap_values_df = pd.DataFrame(shap_values, columns=X.columns, index=X.index)

    
        if encoder not in encoders_1to1:
            for mapping in enc.mapping:
                shap_values_df[mapping['col']]=shap_values_df.loc[:, mapping['mapping'].columns].sum(axis=1)
                shap_values_df.drop(mapping['mapping'].columns, axis=1, inplace=True)
    
    if shapley_library=='acv':
        #fit the ACV model
        X_train = pickle.load(open(path_in + 'X_train.pkl','rb'))
        
        acvtree = ACVTree(model=model, data=X_train.values, cache_normalized=True)

        if encoder in encoders_1to1:
            shap_values_acv = acvtree.py_shap_values(X.values)
        else:
            shap_values_acv = acvtree.shap_values(X.values, C=enc_list_col_ids)
    
        #transforming for the ACV model
        shap_values_df = pd.DataFrame(shap_values_acv[:, :, 0], columns=X.columns, index=X.index)
            
        if encoder not in encoders_1to1:
            for mapping in enc.mapping:
                shap_values_df[mapping['col']] =  shap_values_df.loc[:, mapping['mapping'].columns[0]]
                shap_values_df.drop(mapping['mapping'].columns, axis=1, inplace=True)
    
    
    #analysis
    if shapley_library == 'shap':
        expected_value = explainer.expected_value
        print(expected_value)
    
        #out of the box graphs
        plt.clf()
        shap.decision_plot(explainer.expected_value, shap_values_df.to_numpy(), X_orig, 
                           feature_names=list(shap_values_df.columns), show=False)
        plt.tight_layout()
        plt.savefig(path_out + 'shap1.jpg')
    
        plt.clf()
        shap.decision_plot(explainer.expected_value, explainer.shap_values(X), X, 
                           feature_names=list(X.columns), show=False)
        plt.tight_layout()
        plt.savefig(path_out + 'shap1a.jpg')
    
    plt.clf()
    shap.summary_plot(shap_values_df.to_numpy(), features=X_display, feature_names=X_display.columns,
                      max_display=X_display.shape[1], show=False)
    plt.tight_layout()
    plt.savefig(path_out + 'shap2.jpg')
    
    plt.clf()
    shap.summary_plot(shap_values_df.to_numpy(), features=X_display, feature_names=X_display.columns,
                      plot_type='bar', max_display=X_display.shape[1], show=False)
    plt.tight_layout()
    plt.savefig(path_out + 'shap3.jpg')
    
    #custom graphs
    
    df_shap = shap_values_df
    df = X_orig
    
    selected = []

    # Make a copy of the input data
    shap_v = pd.DataFrame(df_shap)
    feature_list = df.columns
    shap_v.columns = feature_list
    df_v = df.copy().reset_index().drop('index',axis=1)
    
    #custom graphs
    plt.clf()
    df_res, one_cat_vars = global_SHAP(shap_v,df_v, metric='inter_to_intra' )
    plt.tight_layout()
    plt.savefig(path_out + 'shap4.jpg')
    
    plt.clf()
    df_res_final, one_cat_vars = global_SHAP(shap_v,df_v, metric='l_max_inter')
    plt.tight_layout()
    plt.savefig(path_out + 'shap5.jpg')
    
    plt.clf()
    SHAP_bars(shap_v,df_v, one_cat_vars, selected, abs_vals=True)
    plt.tight_layout()
    plt.savefig(path_out + 'shap6.jpg')
    
    plt.clf()
    SHAP_bars(shap_v,df_v, one_cat_vars, selected, abs_vals=False)
    plt.tight_layout()
    plt.savefig(path_out + 'shap7.jpg')
    
    plt.clf()
    df_res, one_cat_vars = global_SHAP(shap_v,df_v, metric='l_max_intra_median')
    plt.tight_layout()
    plt.savefig(path_out + 'shap8.jpg')
    
    #global importance and separability
    
    df_importance = df_res_final[['Variable','l_max_inter', 'inter_to_intra']]
    s=df_importance['l_max_inter'].sum()
    df_importance['l_max_inter_norm']=df_importance['l_max_inter']/s

    df_importance = df_importance.sort_values(by='l_max_inter', ascending=False).reset_index(drop=True)
    
    #plotting individual variables: categories per variable
    ordered_variables = list(df_importance['Variable'])
    
    img_ind = 90 
    for variable in ordered_variables:
        plt.clf()
        show_categories(variable, df_v, shap_v)
        #plt.tight_layout()
        plt.savefig(path_out + 'shap' + str(img_ind) + '.jpg')
        
        img_ind += 1
        
    #save pickle and csv
    pickle.dump(df_importance, open(path_out + 'importance.pkl','wb'))
                
    df_importance.columns = df_importance.columns.str.replace('_',' ')
    df_importance['Variable'] = df_importance['Variable'].str.replace('_',' ')
    df_importance.round(3).to_csv(path_out + 'importance.csv', index=False)
    
    return(df_importance)

In [ ]:
encoders_1to1 = ['None','CatBoostEncoder'] 

In [ ]:
path_in = path + 'results/m' + str(m) + '/'

for library in ['acv', 'shap']:
    for dataset in ['train', 'test']:

        df_results = pd.DataFrame()

        for algorithm in algorithms:
            for encoder in encoders:

                if not((algorithm in ('LinearRegression','CatBoostRegressor') and library=='acv') 
                       or (encoder=='None' and algorithm!= 'CatBoostRegressor')):

                    print(library, algorithm, encoder, dataset)

                    df_importance = shap(algorithm, encoder, path=path_in, dataset=dataset, 
                                         shapley_library=library)
                    df_importance['algorithm'] = algorithm
                    df_importance['encoder'] = encoder

                    df_results = df_results.append(df_importance)


        path_out_global = path_in + 'output/' + library +'/'+ dataset + '/'

        pickle.dump(df_results, open(path_out_global + 'df_importance_all.pkl','wb'))

        df_results.columns = df_results.columns.str.replace('_',' ')
        df_results.round(3).to_csv(path_out_global + 'importance_all.csv', index=False)